In [3]:
!pip install bardapi
!pip install requests
!pip install elasticsearch==7.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 8.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.3
    Uninstalling urllib3-2.0.3:
      Successfully uninstalled urllib3-2.0.3


# linked with blog
If you have a personal blog, you need a login api.

If not you can use sample blog project in https://github.com/HyoungSooo/Django-Blog

in that project have create post api, user create api, user login api using JWT

All of the code below is from the above blog project.

If you want to use your personal blog you need to modify the code.

# create_post
This function automatically creates and posts content on given topics.

### Parameters
* topics:str ('A,B,C')

Each topic must be separated by commas.(',')

* date:datetime ("%Y-%m-%dT%H:%M:%S.%fZ")

The datetime format must be %Y-%m-%dT%H:%M:%S.%fZ

* tags:str ('A,B,C')

Each tags must be separated by commas.(',')

### Modify this function to fit your own blog
I have commented below in detail, so please refer to it and correct it.

In [ ]:
# bard answer
from bardapi import Bard
# bard api config.
# you have to see this project to modify bard config https://github.com/dsdanielpark/Bard-API
api_key = '<api-key>'
bard = Bard(token=api_key, timeout=60)
'\n\n'.join(bard.get_answer(f"how can i use elasticsearch much better")['content'].split('\n\n')[1:-1])

In [ ]:
from elasticsearch import Elasticsearch
# defualt mappings in elasticsearch
# you can change to fit your database
index_mappings = {
  "mappings" : {
    "properties" : {
      "publish" : {
        "type" : "date"
      },
      "created" : {
        "type" : "date"
      },
      "title" : {
        "type" : "text",
        "fields" : {
          "keyword" : {
            "type" : "keyword",
            "ignore_above" : 256
          }
        }
      },
      "body" : {
        "type" : "text",
      },
        "tags" : {
        "type" : "text",
        "fields" : {
          "keyword" : {
            "type" : "keyword",
            "ignore_above" : 256
          }
        }
      }
    },
      
    }
}
index = 'post'

es = Elasticsearch('http://elasticsearch:9200')

if es.indices.exists(index=index):
    pass
else:
    es.indices.create(index=index, body=index_mappings)

In [6]:
import json
import requests
from bardapi import Bard
from elasticsearch import Elasticsearch


# bard api config.
# you have to see this project to modify bard config https://github.com/dsdanielpark/Bard-API
api_key = '<api-key>'
bard = Bard(token=api_key, timeout=60)

def create_post(value, date, tags):
    # change here for your blog urls
    url = 'https://port-0-djangoblog-1duf6sd2blhjy1sm5.sel4.cloudtype.app/'
    topics = value.split(',')
    # change to your blog account
    payload = {
      "password": "example",
      "username": "example"
    }
    res = requests.post(url + 'api/auth/login', json=payload)
    token =  json.loads(res.content).get('token')
    headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {token}'}
    
    body = []
    for i in topics:
        string = f'### {i}\n'
        body.append(string + '\n\n'.join(bard.get_answer(f"{i}")['content'].split('\n\n')[1:-1]))
    
    data = {
      "title": topics[0],
      "slug": 4,
      "body": '\n'.join(body),
      "publish": date,
      "created": date,
      "tags": tags
    }
    
    res = requests.post(url + 'api/math/post', json=data, headers = headers)
    # post to elasticsearch to backup data and search faster
    if res.status_code == 200:
        data.pop('slug', None)
        
        es.index(index=index, doc_type="_doc", body=data)
    
    return res

In [ ]:
import json
import requests
from bardapi import Bard
from elasticsearch import Elasticsearch

# bard api config.
# you have to see this project to modify bard config https://github.com/dsdanielpark/Bard-API
api_key = '<api-key>'
bard = Bard(token=api_key, timeout=60)

def refactoring_post(content, date, tags, title):
    # change here for your blog urls
    url = 'https://port-0-djangoblog-1duf6sd2blhjy1sm5.sel4.cloudtype.app/'
    # change to your blog account
    payload = {
      "password": "example",
      "username": "example"
    }
    res = requests.post(url + 'api/auth/login', json=payload)
    token =  json.loads(res.content).get('token')
    headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {token}'}
    


    body = bard.get_answer(f"{content} refactoring this content")['content']
    
    data = {
      "title": title,
      "slug": 4,
      "body": body,
      "publish": date,
      "created": date,
      "tags": tags
    }
    
    res = requests.post(url + 'api/math/post', json=data, headers = headers)
    # post to elasticsearch to backup data and search faster
    if res.status_code == 200:
        if es.indices.exists(index=index):
            pass
        else:
            es.indices.create(index=index, body=index_mappings)
        
        data.pop('slug', None)
        
        es.index(index=index, doc_type="_doc", body=data)
    
    return res
    